In [21]:
import requests
import pandas as pd
import csv

pd.options.display.max_columns = 50

In [2]:
base_url = 'https://api.cartolafc.globo.com/'

In [3]:
def get_mercado():
    url = "{}atletas/mercado".format(base_url)
    response = requests.get(url)
    return response.json()

In [4]:
def mercado_json_to_df(json):
    #     "atletas":[
    #     {
    #         "nome":"Rodrigo Baldasso da Costa",
    #         "apelido":"Rodrigo",
    #         "foto":"https://s.glbimg.com/es/sde/f/2017/06/13/a5cb57a41ef2b2308c98b76ba24b430a_FORMATO.png",
    #         "atleta_id":37644,
    #         "rodada_id":12,
    #         "clube_id":303,
    #         "posicao_id":3,
    #         "status_id":2,
    #         "pontos_num":5.2,
    #         "preco_num":9.04,
    #         "variacao_num":1.31,
    #         "media_num":3.17,
    #         "jogos_num":8,
    #         "scout":{"CA":2,"CV":1,"FC":5,"FD":2,"FF":5,"FS":15,"I":1,"PE":14,"RB":8,"SG":3}
    #     },
    data = []
    columns = ["atleta_id", "rodada_id", "clube_id", "posicao_id", "status_id",
               "pontos_num", "preco_num", "variacao_num", "media_num", "jogos_num",
               "FS", "PE", "A", "FT", "FD", "FF", "G", "I", "PP",
               "RB", "FC", "GC", "CA", "CV", "SG", "DD", "DP", "GS"]
    scouts = ["FS", "PE", "A", "FT", "FD", "FF", "G", "I", "PP",
               "RB", "FC", "GC", "CA", "CV", "SG", "DD", "DP", "GS"]
    
    for atleta in json["atletas"]:
        row = []
        for column in columns:
            if column in scouts:
                if column in atleta["scout"]:
                    row.append(atleta["scout"][column])
                else:
                    row.append(None)
            else:
                if column in atleta:
                    row.append(atleta[column])
                else:
                    row.append(None)
        data.append(row)
    
    df = pd.DataFrame(data=data, columns=columns)
    return df

In [23]:
def atletas_json_to_df(json):
    #     "atletas":[
    #     {
    #         "nome":"Rodrigo Baldasso da Costa",
    #         "apelido":"Rodrigo",
    #         "foto":"https://s.glbimg.com/es/sde/f/2017/06/13/a5cb57a41ef2b2308c98b76ba24b430a_FORMATO.png",
    #         "atleta_id":37644,
    #         "rodada_id":12,
    #         "clube_id":303,
    #         "posicao_id":3,
    #         "status_id":2,
    #         "pontos_num":5.2,
    #         "preco_num":9.04,
    #         "variacao_num":1.31,
    #         "media_num":3.17,
    #         "jogos_num":8,
    #         "scout":{"CA":2,"CV":1,"FC":5,"FD":2,"FF":5,"FS":15,"I":1,"PE":14,"RB":8,"SG":3}
    #     },
    data = []
    columns = ["atleta_id", "nome", "apelido"]
    
    for atleta in json["atletas"]:
        row = []
        for column in columns:
            if column in atleta:
                if column != "atleta_id":
                    row.append(atleta[column].encode('utf-8'))
                else:
                    row.append(atleta[column])
            else:
                row.append(None)
        data.append(row)
    
    df = pd.DataFrame(data=data, columns=columns)
    return df

In [36]:
scouts_columns = ["FS", "PE", "A", "FT", "FD", "FF", "G", "I", "PP",
                  "RB", "FC", "GC", "CA", "CV", "SG", "DD", "DP", "GS"]

In [13]:
mercado = get_mercado()

scouts = pd.read_csv("../2017/scouts.csv")
scouts_rodada = mercado_json_to_df(mercado)
scouts_rodada[scouts_columns] = scouts_rodada[scouts_columns].fillna(0).astype("int")
new_scouts = pd.concat([scouts, scouts_rodada])
new_scouts.to_csv("../2017/new_scouts.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

scouts_diff = new_scouts.copy()
scouts_cumsum_columns = ["{}_cumsum".format(scout) for scout in scouts_columns]
new_columns = scouts.columns.tolist()[0:10] + scouts_cumsum_columns
scouts_diff.columns = new_columns
scouts_diff[scouts_columns] = scouts_diff.groupby("atleta_id")[scouts_cumsum_columns] \
                                .transform(pd.Series.diff).fillna(0).astype("int")
scouts_diff = scouts_diff.drop(scouts_cumsum_columns, axis=1)
scouts_diff.to_csv("../kaggle/2017_scouts.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

atletas = pd.read_csv("../2017/atletas.csv")
atletas_rodada = atletas_json_to_df(mercado)
new_atletas = pd.merge(atletas, atletas_rodada, how="outer")
new_atletas.to_csv("../2017/new_atletas.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
new_atletas.to_csv("../kaggle/2017_atletas.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)